In [ ]:
from typing import Union, List
import numpy as np

In [ ]:
def walk(atoms: int,
         timesteps: np.ndarray,
         jump_size: int = 1,
         seed: np.random.mtrand.RandomState = np.random.RandomState()) -> np.ndarray:
    """
    Perform a random walk.

    :param atoms: number of atoms
    :param timesteps: the timestep values
    :param jump_size: size of jump
    :param seed: random seed source
    :return: cumulative sum of steps for walk
    """
    possible_moves = np.zeros((6, 3))
    j = 0
    for i in range(0, 6, 2):
        possible_moves[i, j] = jump_size
        possible_moves[i + 1, j] = -jump_size
        j += 1
    choices = seed.choice(len(range(len(possible_moves))), size=(atoms, len(timesteps)))
    steps = np.zeros((atoms, len(timesteps), 3))
    for i in range(steps.shape[0]):
        for j in range(steps.shape[1]):
            steps[i, j] = possible_moves[choices[i, j]]
    return steps

In [ ]:
n_atoms = 128
dt = np.arange(1, 129, 1)

In [ ]:
dt

In [ ]:
cum_steps.shape

In [ ]:
cum_steps = np.cumsum(steps, axis=1)

In [ ]:
disp_3d = []
n_i = np.array([])
for i, n in enumerate(dt):
    disp = np.concatenate([cum_steps[:, np.newaxis, i],
                            np.subtract(cum_steps[:, i + 1:], cum_steps[:, :-(i + 1)])],
                            axis=1)
    disp_3d.append(disp)
    n_i = np.append(n_i, dt[-1] / n * n_atoms)

In [ ]:
disp_3d[50].shape

In [ ]:
dt[50]

In [ ]:
from tqdm import tqdm

In [ ]:
msd = []
for j in tqdm(range(4096)):
    steps = walk(n_atoms, dt, seed=np.random.RandomState(j))
    cum_steps = np.cumsum(steps, axis=1)
    disp_3d = []
    n_i = np.array([])
    for i, n in enumerate(dt):
        disp = np.concatenate([cum_steps[:, np.newaxis, i],
                                np.subtract(cum_steps[:, i + 1:], cum_steps[:, :-(i + 1)])],
                                axis=1)
        disp_3d.append(disp)
        n_i = np.append(n_i, dt[-1] / n * n_atoms)
    msd.append(np.mean(np.sum(disp_3d[-1] ** 2, axis=-1)))

In [ ]:
np.sum(disp_3d[-1] ** 2, axis=1)

In [ ]:
np.sum(disp_3d[-1] ** 2, axis=-2)

In [ ]:
dt[63]

In [ ]:
from scipy.stats import chi2

In [ ]:
f = chi2.fit(msd, floc=0)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(msd, bins=100, density=True)
x = np.linspace(np.min(msd), np.max(msd), 1000)
plt.plot(x, chi2(*f).pdf(x), 'k-')

In [ ]:
f